In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from models import DCGAN, make_losses

In [3]:
from preprocess_data import make_dataset, get_all_image_filenames

In [4]:
fnames = get_all_image_filenames()
N=len(fnames)
print("%d images found" % N)

6033 images found


In [5]:
epochs=50
batch_size=128

In [6]:
image_set = make_dataset(fnames)
tr_set = image_set.shuffle(N).repeat(epochs).batch(batch_size)
tr_feed = tr_set.make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [7]:
noise_dim=100

In [8]:
noise_shape=(batch_size, noise_dim)

In [9]:
birdgan = DCGAN()
generator = birdgan.build_generator(input_shape=(noise_dim,))
discriminator = birdgan.build_discriminator()

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
z = tf.random_uniform(noise_shape)


In [13]:
losses = make_losses(z, real_data, generator, discriminator)

NameError: name 'real_data' is not defined